# RESEARCH NOTEBOOK --> RAJ_REVERSION

In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
import pandas as pd
import pandas_ta as ta  # noqa: F401
from core.data_sources import CLOBDataSource

# Initialize the data source
clob = CLOBDataSource()

In [31]:
# Define the parameters
exchange = "binance_perpetual"
trading_pair = "DOT-USDT"
timeframe = "1h"
days = 120

In [32]:
# Get the candles
candles = await clob.get_candles_last_days(
    exchange, trading_pair, timeframe, days, from_trades=False)

2024-12-06 14:03:56,109 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x33e1d4f40>
2024-12-06 14:03:56,110 - asyncio - ERROR - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x33e1c3b20>, 83303.896803958)])']
connector: <aiohttp.connector.TCPConnector object at 0x33e1d4e80>


In [33]:
candles.plot(type="candles")

In [34]:
import numpy as np


import numpy as np
import pandas as pd

import numpy as np

def calculate_pivots(df: pd.DataFrame, source_column: str, left: int, right: int):
    """
    Calculate pivot highs and lows based on a source column.
    
    Args:
        df (pd.DataFrame): DataFrame containing the price data
        source_column (str): Column name to use for pivot calculations
        left (int): Number of bars to look left
        right (int): Number of bars to look right
    
    Returns:
        tuple: (pivot_highs_idx, pivot_highs, pivot_lows_idx, pivot_lows)
            - pivot_highs_idx: List of datetime indices where pivot highs occur
            - pivot_highs: List of high prices at pivot high points
            - pivot_lows_idx: List of datetime indices where pivot lows occur
            - pivot_lows: List of low prices at pivot low points
    """
    pivot_highs_index = []
    pivot_lows_index = []
    
    # Find pivot points
    for i, value in enumerate(df[source_column]):
        left_index = max(0, i-left)
        right_index = min(len(df), i+right+1)
        range_values = df.iloc[left_index:right_index][source_column]
        
        if value == range_values.max():
            pivot_highs_index.append(i)
        if value == range_values.min():
            pivot_lows_index.append(i)
    
    # Convert indices to datetime and get corresponding prices
    pivot_highs_idx = [df.iloc[i].name for i in pivot_highs_index]
    pivot_lows_idx = [df.iloc[i].name for i in pivot_lows_index]
    pivot_highs = [df.iloc[i][source_column] for i in pivot_highs_index]
    pivot_lows = [df.iloc[i][source_column] for i in pivot_lows_index]
    
    return pivot_highs_idx, pivot_highs, pivot_lows_idx, pivot_lows

# Mean Reversion Strategy Parameters
length = 80  # Length input 
offset = 0.85  # Offset input
sigma = 16  # Sigma input

# Pivot Parameters
left = 7  # Left pivot input
right = 7  # Right pivot input
array_percent = 86  # Percentile threshold

# ALMA over mean Parameters
window_size2 = 9  # Second ALMA length
offset2 = 0.85  # Second ALMA offset  
sigma3 = 16  # Second ALMA sigma

# Add indicators
candles_df = candles.data

# Calculate ALMA
candles_df["alma"] = candles_df.ta.alma(length=length, offset=offset, sigma=sigma)

# Calculate source based on conditions
src = np.where(
    ((candles_df["high"] - candles_df["alma"]) > abs(candles_df["low"] - candles_df["alma"])) & 
    ((candles_df["high"] - candles_df["open"]) > abs(candles_df["low"] - candles_df["open"])),
    candles_df["high"],
    np.where(
        ((candles_df["high"] - candles_df["alma"]) < abs(candles_df["low"] - candles_df["alma"])) &
        ((candles_df["high"] - candles_df["open"]) < abs(candles_df["low"] - candles_df["open"])),
        candles_df["low"],
        candles_df["close"]
    )
)

# Calculate percentage difference from ALMA using the source
candles_df["diff"] = (src - candles_df["alma"]) * 100 / candles_df["alma"]

pivot_highs_idx, pivot_highs, pivot_lows_idx, pivot_lows = calculate_pivots(
    df=candles_df,
    source_column="diff",
    left=left,
    right=right
)

# per_98th = container.percentile_nearest_rank(array_percent)
# plot(display_mean_reversion ? per_98th : na, '', color.lime)
# plot(display_mean_reversion ? -per_98th : na, '', color.lime)
container = pivot_highs + pivot_lows
per_98th = np.percentile(container, array_percent)
print(per_98th)

# # Calculate second ALMA on diff
# candles_df["alma_over_mean"] = candles_df.ta.alma(candles_df["diff"], length=window_size2, offset=offset2, sigma=sigma3)

# # Calculate crossovers
# candles_df["crossunder"] = candles_df.ta.crossunder(candles_df["diff"], candles_df["alma_over_mean"])
# candles_df["crossover"] = candles_df.ta.crossover(candles_df["diff"], candles_df["alma_over_mean"])

# Show last 5 rows of key columns
# candles_df[["timestamp", "close", "alma", "diff", "alma_over_mean", "pivot_low", "pivot_high", "crossunder", "crossover"]].tail(5)
# candles_df[["timestamp", "close", "alma", "diff", "pivot_low", "pivot_high"]].tail(5)

10.013130776381374


In [35]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, 
                    subplot_titles=(trading_pair, 'Pivot Highs and Lows'),
                    row_heights=[0.7, 0.3])
# plot the pivot highs and lows over the candles chart


fig.add_trace(candles.candles_trace(), row=1, col=1)

# plot over the second subplot
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["alma"], mode="lines", line=dict(color="lime", width=2), name="ALMA"), row=2, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df["diff"], mode="lines", line=dict(color="lime", width=2), name="Diff"), row=2, col=1)
fig.add_trace(go.Scatter(x=pivot_highs_idx, y=pivot_highs, mode="markers", marker=dict(color="red", size=10), name="Pivot Highs"), row=2, col=1)
fig.add_trace(go.Scatter(x=pivot_lows_idx, y=pivot_lows, mode="markers", marker=dict(color="green", size=10), name="Pivot Lows"), row=2, col=1)

# remove range slider
fig.update_layout(xaxis_rangeslider_visible=False,
                  height=800,
                  width=1200)
fig.update_layout(title=f'{exchange} - {trading_pair} - {timeframe}')
fig.show()


In [18]:

for i in range(left, len(candles_df) - right):
    left_index = max(0, i-left)
    right_index = min(len(candles_df), i+right+1)
    print(candles_df.iloc[left_index:right_index]["diff"])


2024-12-06 13:35:50,815 - root - ERROR - [<class 'decimal.InvalidOperation'>]


timestamp
2024-08-08 17:00:00   0
2024-08-08 18:00:00   0
2024-08-08 19:00:00   0
2024-08-08 20:00:00   0
2024-08-08 21:00:00   0
2024-08-08 22:00:00   0
2024-08-08 23:00:00   0
2024-08-09 00:00:00   0
2024-08-09 01:00:00   0
2024-08-09 02:00:00   0
2024-08-09 03:00:00   0
2024-08-09 04:00:00   0
2024-08-09 05:00:00   0
2024-08-09 06:00:00   0
2024-08-09 07:00:00   0
Name: diff, dtype: float64
timestamp
2024-08-08 18:00:00   0
2024-08-08 19:00:00   0
2024-08-08 20:00:00   0
2024-08-08 21:00:00   0
2024-08-08 22:00:00   0
2024-08-08 23:00:00   0
2024-08-09 00:00:00   0
2024-08-09 01:00:00   0
2024-08-09 02:00:00   0
2024-08-09 03:00:00   0
2024-08-09 04:00:00   0
2024-08-09 05:00:00   0
2024-08-09 06:00:00   0
2024-08-09 07:00:00   0
2024-08-09 08:00:00   0
Name: diff, dtype: float64
timestamp
2024-08-08 19:00:00   0
2024-08-08 20:00:00   0
2024-08-08 21:00:00   0
2024-08-08 22:00:00   0
2024-08-08 23:00:00   0
2024-08-09 00:00:00   0
2024-08-09 01:00:00   0
2024-08-09 02:00:00   0
2024

TypeError: object of type 'NoneType' has no len()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, 
                    subplot_titles=(trading_pair, 'MACD'),
                    row_heights=[0.7, 0.3])

# Add candlestick
fig.add_trace(go.Candlestick(x=candles_df.index,
                             open=candles_df['open'],
                             high=candles_df['high'],
                             low=candles_df['low'],
                             close=candles_df['close'],
                             name='OHLC'),
              row=1, col=1)

# Add Bollinger Bands
bb_upper = f'BBU_{bb_length}_{bb_std}'
bb_lower = f'BBL_{bb_length}_{bb_std}'
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[bb_upper], 
                         line=dict(color='rgba(173, 204, 255, 0.7)'),
                         name='BB Upper'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[bb_lower], 
                         line=dict(color='rgba(173, 204, 255, 0.7)'),
                         fill='tonexty', fillcolor='rgba(173, 204, 255, 0.1)',
                         name='BB Lower'), row=1, col=1)

# Add MACD
macd = f'MACD_{macd_fast}_{macd_slow}_{macd_signal}'
macd_s = f'MACDs_{macd_fast}_{macd_slow}_{macd_signal}'
macd_hist = f'MACDh_{macd_fast}_{macd_slow}_{macd_signal}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd], 
                         line=dict(color='#00FFFF', width=2),
                         name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd_s], 
                         line=dict(color='#FFA500', width=2),
                         name='Signal'), row=2, col=1)
fig.add_trace(go.Bar(x=candles_df.index, y=candles_df[macd_hist], name='Histogram',
                     marker_color=candles_df[macd_hist].apply(
                         lambda x: '#00FF00' if x >= 0 else '#FF0000')),
              row=2, col=1)

# Update layout for dark theme
fig.update_layout(
    title=f'{exchange} - {trading_pair} - {timeframe}',
    width=1200, height=800,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='MACD', showgrid=False),
    showlegend=False
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

# Show the plot
fig.show()

In [ ]:
# Generate signal

candles_df["signal"] = 0
bbp = candles_df[f"BBP_{bb_length}_{bb_std}"]
macdh = candles_df[f"MACDh_{macd_fast}_{macd_slow}_{macd_signal}"]
macd = candles_df[f"MACD_{macd_fast}_{macd_slow}_{macd_signal}"]

long_condition = (bbp < bb_long_threshold) & (macdh > 0) & (macd < 0)
short_condition = (bbp > bb_short_threshold) & (macdh < 0) & (macd > 0)

candles_df.loc[long_condition, "signal"] = 1
candles_df.loc[short_condition, "signal"] = -1

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                    subplot_titles=('OHLC with BB', 'MACD', 'Signal'),
                    row_heights=[0.6, 0.2, 0.2])

# Add candlestick
fig.add_trace(go.Candlestick(x=candles_df.index,
                             open=candles_df['open'],
                             high=candles_df['high'],
                             low=candles_df['low'],
                             close=candles_df['close'],
                             name='Candlesticks'),
              row=1, col=1)

# Add Bollinger Bands
bb_upper = f'BBU_{bb_length}_{bb_std}'
bb_lower = f'BBL_{bb_length}_{bb_std}'
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[bb_upper], 
                         line=dict(color='rgba(173, 204, 255, 0.7)'),
                         name='BB Upper'), row=1, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[bb_lower], 
                         line=dict(color='rgba(173, 204, 255, 0.7)'),
                         fill='tonexty', fillcolor='rgba(173, 204, 255, 0.1)',
                         name='BB Lower'), row=1, col=1)

# Add MACD
macd_line = f'MACD_{macd_fast}_{macd_slow}_{macd_signal}'
signal_line = f'MACDs_{macd_fast}_{macd_slow}_{macd_signal}'
hist_line = f'MACDh_{macd_fast}_{macd_slow}_{macd_signal}'

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd_line], 
                         line=dict(color='#00FFFF', width=2),
                         name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[signal_line], 
                         line=dict(color='#FFA500', width=2),
                         name='Signal'), row=2, col=1)
fig.add_trace(go.Bar(x=candles_df.index, y=candles_df[hist_line], name='Histogram',
                     marker_color=candles_df[hist_line].apply(
                         lambda x: '#00FF00' if x >= 0 else '#FF0000')),
              row=2, col=1)

# Add the signal line
fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df['signal'],
                         mode='lines',
                         name='Signal',
                         line=dict(color="white")),
              row=3, col=1)

# Update layout for dark theme
fig.update_layout(
    title=f'{exchange} - {trading_pair} - {timeframe}',
    width=1200, height=800,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='MACD', showgrid=False),
    yaxis3=dict(title='Signal', showgrid=False),
    showlegend=False
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

# Show the plot
fig.show()


# CONCLUSION

In this notebook, we have implemented a strategy combining the MACD (Moving Average Convergence Divergence) indicator with Bollinger Bands. We've visualized these indicators along with the price data and generated signals based on their interactions. This approach provides a solid foundation for our trading strategy.
 
## Key components of our strategy include:
 1. MACD for trend identification
 2. Bollinger Bands for volatility measurement and potential reversal points
 3. A signal line derived from the combination of these indicators
 
 The next step is to backtest this strategy to evaluate its profitability and robustness. For this purpose, we have created a controller file named `macd_bb.py` in this folder. This file implements the logic we've developed here, allowing us to conduct comprehensive backtests in the subsequent notebook.